In [ ]:
from flask import Flask, render_template, jsonify
import yfinance as yf
from datetime import datetime

app = Flask(__name__)

# --- MIA-13 EXACT LEDGER CONFIGURATION ---
# Based on deployment date of Feb 13, 2026
LEDGER_DEPLOYS = {
    'NVDA': {'shares': 100, 'cost': 182.81},
    'PLTR': {'shares': 100, 'cost': 131.41},
    'MSFT': {'shares': 100, 'cost': 401.32},
    'AMD':  {'shares': 100, 'cost': 207.32},
    'MU':   {'shares': 100, 'cost': 411.66},
    'QCOM': {'shares': 100, 'cost': 140.70},
    'WMT':  {'shares': 100, 'cost': 133.89},
    'LLY':  {'shares': 100, 'cost': 1040.00},
    'COST': {'shares': 100, 'cost': 1018.48},
    'XLE':  {'shares': 100, 'cost': 54.35},
    'GLD':  {'shares': 100, 'cost': 462.62},
    'XLB':  {'shares': 100, 'cost': 90.50}
}

LATINUM_STASIS = 2939.44  # Cash reserve
INITIAL_BASIS = 430964.24 # Total portfolio value on Feb 13

@app.route('/')
def index():
    return render_template('dashboard3.html')

@app.route('/api/data')
def get_data():
    tickers = list(LEDGER_DEPLOYS.keys())
    # Download 1-minute interval live prices
    data = yf.download(tickers, period='1d', interval='1m')['Close'].iloc[-1]
    
    current_stock_value = 0
    ship_stats = []

    for ticker, meta in LEDGER_DEPLOYS.items():
        live_price = float(data[ticker])
        total_val = live_price * meta['shares']
        current_stock_value += total_val
        
        ship_stats.append({
            'ticker': ticker,
            'price': round(live_price, 2),
            'cost': meta['cost'],
            'notional': round(total_val, 2),
            'gain': round(total_val - (meta['cost'] * meta['shares']), 2)
        })

    total_live_portfolio = current_stock_value + LATINUM_STASIS
    total_return_pct = ((total_live_portfolio - INITIAL_BASIS) / INITIAL_BASIS) * 100

    return jsonify({
        'timestamp': datetime.now().strftime("%H:%M:%S"),
        'total_value': round(total_live_portfolio, 2),
        'total_return': round(total_return_pct, 2),
        'ships': ship_stats
    })

if __name__ == '__main__':
    app.run(debug=True, port=5000,use_reloader=False )


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Feb/2026 11:24:58] "GET / HTTP/1.1" 200 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:24:59] "GET /api/data HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2026 11:25:01] "GET /favicon.ico HTTP/1.1" 404 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:25:14] "GET /api/data HTTP/1.1" 200 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:25:29] "GET /api/data HTTP/1.1" 200 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:25:44] "GET /api/data HTTP/1.1" 200 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:25:59] "GET /api/data HTTP/1.1" 200 -
[*********************100%***********************]  12 of 12 completed
127.0.0.1 - - [18/Feb/2026 11:26:14] "GET /api

In [ ]:
from flask import Flask, render_template, jsonify
import yfinance as yf
import pandas as pd
from datetime import datetime

app = Flask(__name__)

# --- LEDGER CONFIGURATION ---
LEDGER = {
    'SIA-V': {'NVDA': 100, 'PLTR': 100, 'MSFT': 100},
    'SIA-Beta': {'AMD': 100, 'MU': 100, 'QCOM': 100},
    'SIA-Omega': {'WMT': 100, 'LLY': 100, 'COST': 100},
    'SIA-Entropy': {'XLE': 100, 'GLD': 100, 'XLB': 100}
}
INITIAL_BASIS = 430964.24
LATINUM_STASIS = 2939.44

def get_tech_stats(ticker):
    # Fetch historical data for RSI/MA calculation
    hist = yf.Ticker(ticker).history(period="3mo")
    if len(hist) < 50: return {"rsi": 0, "ma20": 0, "ma50": 0}
    
    # Simple Moving Averages
    ma20 = hist['Close'].rolling(window=20).mean().iloc[-1]
    ma50 = hist['Close'].rolling(window=50).mean().iloc[-1]
    
    # RSI Calculation
    delta = hist['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs.iloc[-1]))
    
    return {"rsi": round(rsi, 2), "ma20": round(ma20, 2), "ma50": round(ma50, 2)}

@app.route('/')
def index():
    return render_template('dashboard4.html')

@app.route('/api/data')
def get_data():
    all_tickers = [t for core in LEDGER.values() for t in core.keys()]
    prices = yf.download(all_tickers, period='1d', interval='1m')['Close'].iloc[-1]
    
    total_val = LATINUM_STASIS
    fleet_data = {}
    ship_stats = []

    for core, tickers in LEDGER.items():
        core_val = 0
        for t, shares in tickers.items():
            price = float(prices[t])
            val = price * shares
            core_val += val
            total_val += val
            
            tech = get_tech_stats(t) # Get technical indicators
            ship_stats.append({
                'ticker': t, 'price': round(price, 2), 'core': core,
                'val': round(val, 2), 'rsi': tech['rsi'], 
                'ma20': tech['ma20'], 'ma50': tech['ma50']
            })
        fleet_data[core] = round(core_val, 2)

    return jsonify({
        'timestamp': datetime.now().strftime("%H:%M:%S"),
        'total_value': round(total_val, 2),
        'total_return': round(((total_val - INITIAL_BASIS) / INITIAL_BASIS) * 100, 2),
        'fleet_summary': fleet_data,
        'ships': ship_stats
    })

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port = 5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
